In [7]:
from pydantic import BaseModel, conlist, validator
from typing import List, Dict, Union, Optional
from datetime import date
import re
from datetime import datetime as dt
#from schemas import *
from icecream import ic
import re
from typing import Dict, Union, List
from datetime import datetime
from pydantic import BaseModel
from enum import Enum
from dataclasses import dataclass
import json

In [8]:
class VotingSummary(BaseModel):
    APTA: Optional[int] = None
    APTS: Optional[int] = None
    APTT: Optional[int] = None
    CSEC: Optional[int] = None
    NOMI: Optional[int] = None
    LEGC: Optional[int] = None  # Only for proportional
    BRAN: Optional[int] = None
    NULO: Optional[int] = None
    TOTC: Optional[int] = None

class Candidate(BaseModel):
    code: Optional[str] = None
    votes: Optional[int] = None
 
class Party(BaseModel):
    PART: Optional[int] = None
    LEGP: Optional[int] = None
    TOTP: Optional[int] = None
    candidates: Dict[str, Candidate] = {}

class Position(BaseModel):
    CARG: Optional[int] = None
    TIPO: Optional[int] = None
    VERC: Optional[int] = None
    summary: VotingSummary = VotingSummary()
    party: List[Party] = []

class VotingData(BaseModel):
    IDEL: Optional[int]= None
    position: List[Position] = []

class SecurityData(BaseModel):
    HASH: Optional[str] = None
    ASSI: Optional[str] = None

class Metadata(BaseModel):
    ORIG: Optional[str] = None
    ORLC: Optional[str] = None
    PROC: Optional[int] = None
    DTPL: Optional[str] = None
    PLEI: Optional[int] = None
    TURN: Optional[int] = None
    FASE: Optional[str] = None
    UNFE: Optional[str] = None
    MUNI: Optional[int] = None
    ZONA: Optional[int] = None
    SECA: Optional[int] = None
    AGRE: List[float] = []
    IDUE: Optional[int] = None
    IDCA: Optional[int] = None
    HIQT: Optional[int] = None
    HICA: Optional[list[str]] = None
    VERS: Optional[str] = None

class Details(BaseModel):
    LOCA: Optional[int] = None
    APTO: Optional[int] = None
    APTS: Optional[int] = None
    APTT: Optional[int] = None
    COMP: Optional[int] = None
    FALT: Optional[int] = None
    HBBM: Optional[int] = None 
    HBBG: Optional[int] = None
    HBSB: Optional[int] = None
    DTAB: Optional[str] = None
    HRAB: Optional[str] = None
    DTFC: Optional[str] = None
    HRFC: Optional[str] = None

class Header(BaseModel):
    QRBU: list[int] = []
    VRQR: Optional[str] = None
    VRCH: Optional[str] = None

class Content(BaseModel):
    metadata: Metadata = Metadata()
    details: Details = Details()
    voting: VotingData = VotingData()
    security: SecurityData = SecurityData()

class BoletimUrna(BaseModel):
    type: str
    finished: bool
    last_carg: int
    last_party: int|None
    open_steps: List[str]|None
    header: Header
    content: Content

class BoletimUrnaFinished(BaseModel):
    evaluator_phone: str
    boletim_urna: BoletimUrna

class ProcessingStep(Enum):
        WAITING: str = "waiting"
        OPEN: str = "open"
        FINISHED: str = "finished"

@dataclass
class ProcessingStatus:
    header: ProcessingStep = ProcessingStep.OPEN
    context_setup: ProcessingStep = ProcessingStep.WAITING
    content: ProcessingStep = ProcessingStep.WAITING
    metadata: ProcessingStep = ProcessingStep.WAITING
    details: ProcessingStep = ProcessingStep.WAITING
    voting: ProcessingStep = ProcessingStep.WAITING
    position: ProcessingStep = ProcessingStep.WAITING
    party: ProcessingStep = ProcessingStep.WAITING
    candidate: ProcessingStep = ProcessingStep.WAITING
    summary: ProcessingStep = ProcessingStep.WAITING
    security: ProcessingStep = ProcessingStep.OPEN

In [35]:
class BulletinUrnaParser:
    def __init__(self, id: int):
        self.id = id
        self.parsing_processing_status = ProcessingStatus()
        self.header = Header()
        self.content = Content()
        self.current_position = None
        self.current_party = None
        self.current_candidates = {}
        self.parsed_bulletin = None
        self.empty_party = False
        self.counters = {
            "header": [3, 0],
            "metadata": [17, 0],
            "details": [13, 0],
            "voting": [2, 0],
            "position": [4, 0],
            "party": [0, 0],
            "candidate": [0, 0],
            "summary": [0, 0],
            "security": [0, 0]
        }
        self.counter = 0

    def _get_last_bu(self):
        # Lazy load from mock DB or real DB
        mock_db = {1: 'bu_big_1.json', 2: 'bu_big_2.json', 3: 'bu_big_3.json'}
        with open(mock_db[self.id]) as f:
            bulletin = BoletimUrna(**json.load(f))
        return bulletin

    def _update_status(self, step:list = None, next_step:list = None):
        if step:
            for s in step:
                self.parsing_processing_status.__setattr__(s, ProcessingStep.FINISHED)
                print(f"Finished processing step: {s}")
        if next_step:
            for s in next_step:
                self.parsing_processing_status.__setattr__(s, ProcessingStep.OPEN)
                print(f"Opened processing step: {s}")

    def _update_counter(self, step):
        self.counters[step][1] += 1
        #if self.counters[step][1] == self.counters[step][0]:
            #self._update_status([step], [self.counters[step][0]])

    def _parse_field(self, key_value, field_mapping, steps_status: list, obj_nickname=str):
        key = key_value[0]
        value = key_value[1:] if len(key_value) > 2 else key_value[1]
        if key in field_mapping and self._validate_open_steps(steps_status):
            attr, is_int, fmt, step, next_step = field_mapping[key]
            value = int(value) if is_int else value
            if fmt:  # If there's formatting to apply (e.g., date formatting)
                value = fmt(value)
    
            obj_nicknames = {
                "header": self.header,
                "content": self.content,
                "position": self.current_position,
                "party": self.current_party,
            }
            obj = obj_nicknames[obj_nickname]
            attr_parts = attr.split(".")
            for part in attr_parts[:-1]:
                obj = getattr(obj, part)
            setattr(obj, attr_parts[-1], value)

            self._update_status(step, next_step)
            return True
        return False
    
    def _code_size(self):
        return "large" if self.header.QRBU[1] > 1 else "small"
    
    def _is_first_code(self):
        return self.header.QRBU[0] == 1
    
    def _is_finished(self):
        return (self.header.QRBU[0] == self.header.QRBU[1]) and self.content.security.ASSI is not None
    
    def _check_continuity(self, last_QRBU, current_QRBU):
        if not last_QRBU[0] + 1 == current_QRBU[0]:
            raise ValueError(f"Boletim não está em sequencia, esperado {last_QRBU[0] + 1}, recebeu {current_QRBU[0]}")
        
    def _set_open_steps_in_context(self, open_steps:list):
        for step in open_steps:
            self.parsing_processing_status.__setattr__(step, ProcessingStep.OPEN)

    def _get_open_steps(self):
        return [k for k, v in self.parsing_processing_status.__dict__.items() if v == ProcessingStep.OPEN]
    
    def _validate_open_steps(self, steps):
        open_steps = self._get_open_steps()
        n_ok = 0
        for step in steps:
            if step in open_steps:
                n_ok += 1
        if n_ok == len(steps):
            return True
        return False
    
    # Parsing Header Fields
    def _parse_header(self, key_value):
        field_mapping = {
            "QRBU": ("QRBU", False, lambda v: [int(k) for k in v], None, None),
            "VRQR": ("VRQR", False, None, None, None),
            "VRCH": ("VRCH", False, None, ["header"], ["context_setup"]),
        }
        return self._parse_field(key_value, field_mapping, ["header", "security"], "header")
    
    def _set_context(self):
        if self.parsing_processing_status.context_setup == ProcessingStep.OPEN:
            if self._code_size() == "large" and not self._is_first_code():
                print("Attempting to continue")
                last_bulletin = self._get_last_bu()
                last_carg = last_bulletin.last_carg
                last_party = last_bulletin.last_party
                open_steps = last_bulletin.open_steps
                print(f"Retrieved last bulletin with last_carg: {last_carg} and last_party: {last_party} and QRBU: {last_bulletin.header.QRBU} to continue parsing")
                self._check_continuity(last_bulletin.header.QRBU, self.header.QRBU)
                last_bulletin.header.QRBU[0] = self.header.QRBU[0] # Update last bulletin QRBU to current QRBU
                self.header = last_bulletin.header
                self.content = last_bulletin.content
                self._update_status(next_step=open_steps)
                # current position is the object that contains CARG == last_carg
                for p in self.content.voting.position:
                    if p.CARG == last_carg:
                        self.current_position = p
                # current party if it is not None is the last_party or where PART == last_party
                if last_party:
                    for p in self.current_position.party:
                        if p.PART == last_party:
                            self.current_party = p
                    if not self.current_party:
                        self.current_party = self.current_position.party[-1]
                    self.current_candidates = self.current_party.candidates
                self._update_status(step=["context_setup"])
            else:
                print("small or first large")
                self._update_status(step=["context_setup"], next_step=["content", "metadata"])
        return False

    # Parsing Metadata Fields
    def _parse_metadata(self, key_value):
        field_mapping = {
            "ORIG": ("metadata.ORIG", False, None, None, None),
            "ORLC": ("metadata.ORLC", False, None, None, None),
            "PROC": ("metadata.PROC", True, None, None, None),
            "DTPL": ("metadata.DTPL", False, lambda v: datetime.strptime(v, '%Y%m%d').strftime('%Y-%m-%d'), None, None),
            "PLEI": ("metadata.PLEI", True, None, None, None),
            "TURN": ("metadata.TURN", True, None, None, None),
            "FASE": ("metadata.FASE", False, None, None, None),
            "UNFE": ("metadata.UNFE", False, None, None, None),
            "MUNI": ("metadata.MUNI", True, None, None, None),
            "ZONA": ("metadata.ZONA", True, None, None, None),
            "SECA": ("metadata.SECA", True, None, None, None),
            "AGRE": ("metadata.AGRE", False, lambda v: [int(k) for k in v.split(".")], None, None),
            "IDUE": ("metadata.IDUE", True, None, None, None),
            "IDCA": ("metadata.IDCA", True, None, None, None),
            "HIQT": ("metadata.HIQT", True, None, None, None),
            "HICA": ("metadata.HICA", False, None, None, None),
            "VERS": ("metadata.VERS", False, None, ["metadata"], ["details"]),
        }
        return self._parse_field(key_value, field_mapping, ["content", "metadata", "security"], "content")

    # Parsing Details Fields
    def _parse_details(self, key_value):
        field_mapping = {
            "LOCA": ("details.LOCA", True, None, None, None),
            "APTO": ("details.APTO", True, None, None, None),
            "APTS": ("details.APTS", True, None, None, None),
            "APTT": ("details.APTT", True, None, None, None),
            "COMP": ("details.COMP", True, None, None, None),
            "FALT": ("details.FALT", True, None, None, None),
            "HBBM": ("details.HBBM", True, None, None, None),
            "HBBG": ("details.HBBG", True, None, None, None),
            "HBSB": ("details.HBSB", True, None, None, None),
            "DTAB": ("details.DTAB", False, lambda v: datetime.strptime(v, '%Y%m%d').strftime('%Y-%m-%d'), None, None),
            "HRAB": ("details.HRAB", False, lambda v: f"{v[:2]}:{v[2:4]}:{v[4:]}", None, None),
            "DTFC": ("details.DTFC", False, lambda v: datetime.strptime(v, '%Y%m%d').strftime('%Y-%m-%d'), None, None),
            "HRFC": ("details.HRFC", False, lambda v: f"{v[:2]}:{v[2:4]}:{v[4:]}", ["details"], ["voting"]),
        }
        return self._parse_field(key_value, field_mapping, ["content", "details", "security"], "content")

    # Parsing Voting Fields
    def _parse_voting(self, key_value):
        field_mapping = {
            "IDEL": ("voting.IDEL", True, None, None, ["position"]),
        }
        return self._parse_field(key_value, field_mapping, ["content", "voting", "security"], "content")

    # Parsing Position Fields
    def _parse_position(self, key_value):
        field_mapping = {
            "CARG": ("CARG", True, None, None, None),
            "TIPO": ("TIPO", True, None, None, None),
            "VERC": ("VERC", True, None, None, ["party"]),
        }
        if key_value[0] == "CARG":
                self.empty_party = True if int(key_value[1]) == 11 else False
                print(f"Empty party: {self.empty_party}")
                if (self.current_position and self.current_position.CARG != int(key_value[1])):
                    print("ATTENTION: New CARG found")
                    # check if the current_position CARG exits in the content.voting.position if not append if yes delete and append
                    if self.current_party:
                        self.current_position.party.append(self.current_party)
                    if not any(p.CARG == self.current_position.CARG for p in self.content.voting.position):
                        self.content.voting.position.append(self.current_position)
                    else:
                        self.content.voting.position = [p for p in self.content.voting.position if p.CARG != self.current_position.CARG]
                        self.content.voting.position.append(self.current_position)
                self.current_position = Position()
                self.current_party = None
                self.current_candidates = {}
        return self._parse_field(key_value, field_mapping, ["content", "voting", "position", "security"], "position")

    # Parsing Party Fields
    def _parse_party(self, key_value):
        field_mapping = {
            "PART": ("PART", True, None, None, ["candidate"]),
        }
        print((self.empty_party and self.parsing_processing_status.candidate == ProcessingStep.WAITING))
        if key_value[0] == "PART" or (self.empty_party and self.parsing_processing_status.candidate != ProcessingStep.OPEN and self.parsing_processing_status.summary != ProcessingStep.OPEN and key_value[0] != "HASH"):
            print("ATTENTION: New PART found")
            if self.empty_party and self.parsing_processing_status.candidate != ProcessingStep.OPEN:
                self._update_status(next_step=["candidate"])
            if self.current_party:
                current_party_first_candidate = list(self.current_party.candidates.keys())[0]
                party_to_delete = None
                for party in self.current_position.party:
                    first_candidate = list(party.candidates.keys())[0]
                    print(f"Current party first candidate: {current_party_first_candidate}, First candidate: {first_candidate}")
                    if current_party_first_candidate == first_candidate:
                        print("ATTENTION: same party found")
                        party_to_delete = party
                if party_to_delete:
                    print("Deleting party")
                    self.current_position.party = [p for p in self.current_position.party if p != party_to_delete]
                self.current_position.party.append(self.current_party)
            print("Creating party")
            self.current_party = Party()
            self.current_candidates = {}
        return self._parse_field(key_value, field_mapping, ["content", "voting", "position", "party", "security"], "party")

    def _is_candidate(self, key_value):
        try:
            int(key_value[0])
            return True
        except:
            return False

    def _verify_next_candidate_exists(self):
        return self.next_part and self._is_candidate(self.next_part.split(":"))

    # Parsing Candidate Fields
    def _parse_candidate(self, key_value):
        if self._is_candidate(key_value) and self._validate_open_steps(["content", "voting", "position", "party", "candidate", "security"]):
            self.current_candidates[key_value[0]] = Candidate(code=key_value[0], votes=int(key_value[1]))
            if not self._verify_next_candidate_exists():
                self._update_status(step=["candidate"], next_step=["summary"])
                self.current_party.candidates = self.current_candidates
                self.current_candidates = {}
            return True
        return False
    
    # Parsing POST Party Fields
    def _parse_post_party(self, key_value):
        field_mapping = {
            "LEGP": ("LEGP", True, None, None, None),
            "TOTP": ("TOTP", True, None, None, None),
        }
        return self._parse_field(key_value, field_mapping, ["content", "voting", "position", "party", "summary", "security"], "party")

    # Parsing Summary Fields
    def _parse_summary(self, key_value):
        field_mapping = {
            "APTA": ("summary.APTA", True, None, None, None),
            "APTS": ("summary.APTS", True, None, None, None),
            "APTT": ("summary.APTT", True, None, None, None),
            "NOMI": ("summary.NOMI", True, None, None, None),
            "BRAN": ("summary.BRAN", True, None, None, None),
            "NULO": ("summary.NULO", True, None, None, None),
            "LEGC": ("summary.LEGC", True, None, None, None),
            "TOTC": ("summary.TOTC", True, None, ["summary"], None),
        }
        return self._parse_field(key_value, field_mapping, ["content", "voting", "position", "party", "summary", "security"], "position")

    # Parsing Security Fields
    def _parse_security(self, key_value):
        field_mapping = {
            "HASH": ("security.HASH", False, None, None, None),
            "ASSI": ("security.ASSI", False, None, ["content", "position", "party", "security"], None),
        }
        if key_value[0] == "HASH":
            if self.current_party:
                self.current_position.party.append(self.current_party)
            if self.current_position:
                if any(p.CARG == self.current_position.CARG for p in self.content.voting.position):
                    self.content.voting.position = [p for p in self.content.voting.position if p.CARG != self.current_position.CARG]
                self.content.voting.position.append(self.current_position)
            if self.next_part == None: 
                self._update_status(next_step=["candidate"])
        return self._parse_field(key_value, field_mapping, ["content", "security"], "content")

    # Main execute function
    def execute(self, bu_string: str):
        parts = re.split(r'\s+', bu_string)
        print(f"Initiating parsing of bulletin")
        print(f"Opened processing step: header")
        print(f"Processing steps with OPEN status on start: {self._get_open_steps()}")
        for part in parts:
            print(f"Counter: {self.counters}")
            key_value = part.split(":")
            print(f"Processing key_value: {key_value}")

            self.next_part = parts[self.counter + 1] if self.counter + 1 < len(parts) else None
            self.counter += 1

            # Parse in order and skip if handled
            if self._parse_header(key_value):
                print(f"executed header")
                self._update_counter("header")
                continue

            elif self._set_context():
                pass

            elif self._parse_metadata(key_value):
                print(f"executed metadata")
                self._update_counter("metadata")
                continue

            elif self._parse_details(key_value):
                print(f"executed details")
                self._update_counter("details")
                continue

            elif self._parse_voting(key_value):
                print(f"executed voting")
                self._update_counter("voting")
                continue

            elif self._parse_position(key_value):
                print(f"executed position")
                self._update_counter("position")
                continue

            elif self._parse_party(key_value):
                print(f"executed party")
                self._update_counter("party")
                continue

            elif self._parse_candidate(key_value):
                print(f"executed candidate")
                self._update_counter("candidate")
                continue

            elif self._parse_post_party(key_value):
                self._update_counter("party")
                continue

            elif self._parse_summary(key_value):
                self._update_counter("summary")
                continue

            elif self._parse_security(key_value):
                self._update_counter("security")
                continue
        
        print("FINISHED PARSING")
        print(f"Processing steps with OPEN status on start: {self._get_open_steps()}")
        
        # At the end, assemble the bulletin
        self.parsed_bulletin = BoletimUrna(
            type=self._code_size(),
            finished=self._is_finished(),
            last_carg=self.current_position.CARG,
            last_party=self.current_party.PART if self.current_party else None,
            open_steps=self._get_open_steps(),
            header=self.header,
            content=self.content
        )
        return self.parsed_bulletin

    def export_json(self, name: str):
        with open(name, 'w') as f:
            json.dump(self.parsed_bulletin.dict(), f, indent=4)

In [36]:
bu_small_string = {"name": "bu_small.json",
                   "content": "QRBU:1:1 VRQR:1.5 VRCH:20240507 ORIG:VOTA ORLC:LEG PROC:1000 DTPL:20241006 PLEI:1100 TURN:1 FASE:S UNFE:AC MUNI:1120 ZONA:8 SECA:2 AGRE:3.4 IDUE:2031032 IDCA:387626604953598569436326 HIQT:1 HICA:1:387626604953598569436326 VERS:9.20.0.0 LOCA:1 APTO:144 APTS:144 APTT:0 COMP:2 FALT:142 HBBM:0 HBBG:0 HBSB:2 DTAB:20241006 HRAB:172113 DTFC:20241006 HRFC:172300 IDEL:1101 CARG:13 TIPO:1 VERC:202405101700 PART:93 93001:1 LEGP:0 TOTP:1 APTA:144 APTS:144 APTT:0 NOMI:1 LEGC:0 BRAN:1 NULO:0 TOTC:2 CARG:11 TIPO:0 VERC:202405101700 92:1 APTA:144 APTS:144 APTT:0 NOMI:1 BRAN:0 NULO:1 TOTC:2 HASH:57D17C50037E7E4C624468438AE77BEA6562076A20CD454FE30EAD413F7D6174ADE59D0D97013BD8F9F50316D766D3670B57FBB7D396C08DD4C4D9250E7B05FC ASSI:B2FA068D49111BA3A61DA0DC44334F8EC41598C73DE90B8E22AA64DAB8C10AA083FD0737B47560B3C6C837D0F24044ABB18DD5A4D2BC66884DB57BFCFA40F906"}

In [37]:
bu_big_1_string = {"name": "bu_big_1.json",
                   "content": "QRBU:1:4 VRQR:1.5 VRCH:20240507 ORIG:VOTA ORLC:LEG PROC:1000 DTPL:20241006 PLEI:1100 TURN:1 FASE:S UNFE:AC MUNI:1392 ZONA:9 SECA:22 IDUE:2033200 IDCA:216820571350570928893711 HIQT:1 HICA:1:216820571350570928893711 VERS:9.21.0.0 LOCA:4 APTO:559 APTS:559 APTT:0 COMP:504 FALT:55 DTAB:20241006 HRAB:091502 DTFC:20241006 HRFC:170042 IDEL:1101 CARG:13 TIPO:1 VERC:202406131529 PART:91 91001:1 91002:1 91003:4 91004:1 91005:3 91006:3 91007:1 91009:1 91010:1 91011:1 91012:3 91013:1 91014:2 91015:2 91018:1 91020:3 91022:5 91024:2 91025:2 91026:3 91027:3 91028:2 91029:1 91030:2 91031:1 91032:1 91033:2 91034:1 91035:1 91036:1 91037:2 91038:2 91039:5 91040:3 91043:1 91044:4 91045:1 91046:3 91047:2 91048:1 91049:2 91050:1 91051:3 91052:2 91054:2 91055:2 91056:3 91057:1 91059:3 LEGP:0 TOTP:99 PART:92 92001:2 92002:3 92003:2 92004:3 92005:2 92006:2 92007:1 HASH:C84EAF7AEC9D9CD157B5F00206C708B0E6BB67226098903B0050F12D3BEFF79B7EF6025FBA2254E388E264511816BC270EB8701FC9455170FD905BFC6E19628"}

In [38]:
bu_big_2_string = {"name": "bu_big_2.json",
                   "content": "QRBU:2:4 VRQR:1.5 VRCH:20240507 92008:6 92009:2 92010:1 92011:2 92013:1 92014:1 92015:5 92017:3 92018:1 92019:4 92020:3 92021:1 92022:1 92023:1 92024:3 92025:1 92026:2 92027:3 92030:4 92032:2 92033:2 92034:2 92035:1 92036:3 92037:2 92038:1 92039:6 92040:1 92041:1 92042:2 92043:4 92044:2 92047:3 92048:4 92049:1 92050:1 92051:1 92053:2 92054:1 92055:3 92056:1 92057:2 92058:2 92059:2 LEGP:0 TOTP:112 PART:93 93001:2 93002:2 93003:2 93005:2 93006:3 93007:2 93008:5 93009:1 93010:1 93011:2 93012:2 93013:3 93014:1 93015:2 93017:3 93018:1 93019:2 93020:3 93022:1 93023:1 93024:1 93025:5 93026:3 93027:2 93028:1 93029:2 93030:2 93031:1 93032:1 93033:4 93034:2 93035:2 93036:1 93037:1 93039:1 93040:2 93041:1 93042:2 93043:1 93044:1 93045:1 93046:2 93047:1 93048:5 93049:2 93050:1 93052:3 93053:4 93054:2 93055:1 93056:2 93057:2 93058:2 93059:1 LEGP:1 HASH:643D452C83DBADB591895D9D2ACEF3F6E706E3A2E67D53671812BADEF002CC0AE9D16C0DC7BD5F2BF9FDA5F6E616B04974276843375B46D15CF88300FECD7D96"}

In [39]:
bu_big_3_string = {"name": "bu_big_3.json",
                   "content": "QRBU:3:4 VRQR:1.5 VRCH:20240507 TOTP:107 PART:94 94001:2 94003:1 94004:3 94005:3 94007:3 94008:1 94010:1 94011:2 94012:2 94013:5 94014:2 94015:2 94017:1 94018:3 94019:1 94020:1 94021:1 94022:1 94023:2 94024:1 94025:2 94026:2 94027:2 94028:2 94029:1 94030:3 94031:2 94032:1 94033:1 94034:3 94035:1 94037:1 94038:3 94039:2 94040:3 94041:1 94042:1 94044:1 94045:1 94046:1 94047:2 94048:1 94050:3 94052:1 94053:1 94054:1 94056:2 94057:2 94058:1 LEGP:1 TOTP:87 PART:95 95001:3 95002:3 95016:1 95017:2 95018:1 95022:1 95023:2 95024:2 95025:1 95026:1 95027:1 95028:2 95029:3 95030:3 95031:3 95032:2 95033:2 95035:1 95036:1 95037:1 95038:2 95040:4 95041:2 95043:2 95044:1 95045:3 95046:3 95047:1 95048:2 95049:3 95050:1 95051:2 95052:1 95053:1 95054:2 95056:1 HASH:FA3D6F3A762B4F03F0813C7AC06E6230017FAB80AE7333098E5255F5879A3F1EEDB126D1AAA5F0188131261961F4F041F4BE06F2961433589EB9AA51FBF8C484"}

In [55]:
bu_big_4_string = {"name": "bu_big_4.json",
                   "content": "QRBU:4:4 VRQR:1.5 VRCH:20240507 95058:1 95059:2 LEGP:3 TOTP:99 APTA:559 APTS:559 APTT:0 NOMI:499 LEGC:5 BRAN:0 NULO:0 TOTC:504 CARG:11 TIPO:0 VERC:202406131529 91:102 92:105 93:111 94:95 95:91 APTA:559 APTS:559 APTT:0 NOMI:504 BRAN:0 NULO:0 TOTC:504 HASH:27FF0E01FB973621CAD76FF624B71A396AA858E5724179A0DA3CC160811F5BF550D85024C75CA54686901FBC12695D21C2EBDA46EA7D1B2593B4459EAF0BDEF6 ASSI:154D5E3ABD3567C353D144A324BE4D2EFBDB716685F3155AB07C2105B3774FCE3262FDCE50CE5FBE95828EC19141991C04FAA0A91A2C54CDC33E6D0716F1190E"}

In [56]:
bu = bu_big_4_string

In [57]:
parser = BulletinUrnaParser(3)
parser.execute(bu["content"])


Initiating parsing of bulletin
Opened processing step: header
Processing steps with OPEN status on start: ['header', 'security']
Counter: {'header': [3, 0], 'metadata': [17, 0], 'details': [13, 0], 'voting': [2, 0], 'position': [4, 0], 'party': [0, 0], 'candidate': [0, 0], 'summary': [0, 0], 'security': [0, 0]}
Processing key_value: ['QRBU', '4', '4']
executed header
Counter: {'header': [3, 1], 'metadata': [17, 0], 'details': [13, 0], 'voting': [2, 0], 'position': [4, 0], 'party': [0, 0], 'candidate': [0, 0], 'summary': [0, 0], 'security': [0, 0]}
Processing key_value: ['VRQR', '1.5']
executed header
Counter: {'header': [3, 2], 'metadata': [17, 0], 'details': [13, 0], 'voting': [2, 0], 'position': [4, 0], 'party': [0, 0], 'candidate': [0, 0], 'summary': [0, 0], 'security': [0, 0]}
Processing key_value: ['VRCH', '20240507']
Finished processing step: header
Opened processing step: context_setup
executed header
Counter: {'header': [3, 3], 'metadata': [17, 0], 'details': [13, 0], 'voting':

BoletimUrna(type='large', finished=True, last_carg=11, last_party=None, open_steps=['voting', 'candidate'], header=Header(QRBU=[4, 4], VRQR='1.5', VRCH='20240507'), content=Content(metadata=Metadata(ORIG='VOTA', ORLC='LEG', PROC=1000, DTPL='2024-10-06', PLEI=1100, TURN=1, FASE='S', UNFE='AC', MUNI=1392, ZONA=9, SECA=22, AGRE=[], IDUE=2033200, IDCA=216820571350570928893711, HIQT=1, HICA=['1', '216820571350570928893711'], VERS='9.21.0.0'), details=Details(LOCA=4, APTO=559, APTS=559, APTT=0, COMP=504, FALT=55, HBBM=None, HBBG=None, HBSB=None, DTAB='2024-10-06', HRAB='09:15:02', DTFC='2024-10-06', HRFC='17:00:42'), voting=VotingData(IDEL=1101, position=[Position(CARG=13, TIPO=1, VERC=202406131529, summary=VotingSummary(APTA=559, APTS=559, APTT=0, CSEC=None, NOMI=499, LEGC=5, BRAN=0, NULO=0, TOTC=504), party=[Party(PART=91, LEGP=0, TOTP=99, candidates={'91001': Candidate(code='91001', votes=1), '91002': Candidate(code='91002', votes=1), '91003': Candidate(code='91003', votes=4), '91004': Ca

In [58]:
# save boletim_urna to json file
parser.export_json(bu["name"])